In [1]:
import pandas as pd
import numpy as np

### Function to read Skeleton (pandas DataFrame) from Kinect raw extracted data with KinectXEFTools (.txt file)

In [3]:
def read_Skeleton(path):
    f=open(path) 
    head=f.readline()
    df=pd.DataFrame()
    
    slice_= lambda A, n=2: [A[i:i+n] for i in range(0, len(A), n)]
    
    c=0
    for line in f:
        x=line.split(',')
        
        l=[]
        ln=[]
        for n,i in enumerate(x):
            if n>8:
                try:
                    int(i)
                    if int(i)==0 and int(x[n+1])==0:
                        pass
                    else:
                        l.append(i)
                        ln.append(n)
                except:pass
        ln_slice=slice_(ln)
        ln_fit=[]
        for n,i in enumerate(ln_slice):
            ln_fit.append([i[0]-n,i[0]-n+2])

        for i in ln_fit:
            x[i[0]:i[1]]=['.'.join(x[i[0]:i[1]])]
            
        ind=[]
        var=[]
        for n,i in enumerate(head.split(',')):
            if 'Rotation' in i:
                pass
            else:
                var.append(i)
                ind.append(n)
        x2=[]
        for n,i in enumerate(x):
            if n in ind:
                x2.append(i)
            else:pass
        
        df[str(c)]=x2
        c+=1
        
    df['Event']=var
    
    return(df)

In [4]:
#EXAMPLE
path='D:\\TeDFES\\kinect_raw\\records\\record2\\record2_Skeleton.txt'
data=read_Skeleton(path)
data

,0,1,2,3,4,5,6,7,8,9,Event
0,95,96,97,98,99,100,101,102,103,104,EventIndex
1,31741321,32071854,32411907,32741618,33072034,33412819,33741730,34071715,34411938,34741177,Time
2,72057594037931485,72057594037931485,72057594037931485,72057594037931485,72057594037931485,72057594037931485,72057594037931485,72057594037931485,72057594037931485,72057594037931485,SkeletonId
3,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,NOT_TRACKED,HandLeftConfidence
4,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,HandLeftState
...,...,...,...,...,...,...,...,...,...,...,...
127,ThumbRight,ThumbRight,ThumbRight,ThumbRight,ThumbRight,ThumbRight,ThumbRight,ThumbRight,ThumbRight,ThumbRight,Joint
128,TRACKED,TRACKED,INFERRED,TRACKED,TRACKED,TRACKED,TRACKED,TRACKED,TRACKED,TRACKED,Status
129,-0.123942,-0.06063346,-0.3129704,-0.4548533,-0.3702823,-0.4517885,-0.3301727,-0.2523216,-0.2832001,-0.1544185,PositionX
130,1.312996,1.399962,1.74456,1.533928,1.503377,1.369797,1.389979,1.318504,1.316367,1.423027,PositionY


### Obtain lists and arrays with Joints coordinates (Ej: AnkleLeft_Z, AnkleLeft_Z_arr)

In [5]:
TimeIndex=[] #Time for each event
for i in data.loc[data['Event']=='Time']:
    try:
        TimeIndex.append(int(data.loc[data['Event']=='Time'][i]))
    except:pass

EventIndex=[] #Index for each event
for i in data.loc[data['Event']=='EventIndex']:
    try:
        EventIndex.append(int(data.loc[data['Event']=='EventIndex'][i]))
    except:pass

#Lists and arrays for each joint in each event
list_datas=[]
c=7 #empiezan los puntos
for i in range(int(len(data['0'][7:])/5)):
    d=data.iloc[c:c+5]
    list_datas.append(d)
    c+=5
    
for df in list_datas:
    
    globals()['{}_X'.format(df['0'].iloc[0])]=[]
    globals()['{}_Y'.format(df['0'].iloc[0])]=[]
    globals()['{}_Z'.format(df['0'].iloc[0])]=[]
    
    
    df1=df.loc[df['Event']=='PositionX']
    for i in df1:
        try:
            globals()['{}_X'.format(df['0'].iloc[0])].append(float(df1[i]))
        except: pass
        
    df2=df.loc[df['Event']=='PositionY']
    for i in df2:
        try:
            globals()['{}_Y'.format(df['0'].iloc[0])].append(float(df2[i]))
        except: pass
        
    df3=df.loc[df['Event']=='PositionZ']
    for i in df3:
        try:
            globals()['{}_Z'.format(df['0'].iloc[0])].append(float(df3[i]))
        except:pass
        
    globals()['{}_X_arr'.format(df['0'].iloc[0])]=np.array(globals()['{}_X'.format(df['0'].iloc[0])])
    globals()['{}_Y_arr'.format(df['0'].iloc[0])]=np.array(globals()['{}_Y'.format(df['0'].iloc[0])])
    globals()['{}_Z_arr'.format(df['0'].iloc[0])]=np.array(globals()['{}_Z'.format(df['0'].iloc[0])])
    

#### Now, we have data of all joints

In [8]:
#EXAMPLE
FootLeft_X_arr

array([-0.1904737, -0.2040588, -0.4253319, -0.1539221, -0.2608739,
       -0.4484123, -0.4232363, -0.4408005, -0.3324264, -0.4158577])

## Reading .dat file with depth data from KinectXEFTools

In [10]:
def read_Depth(path_depth):
    #DEPTH IMAGES HAVE RESOLUTION 424X512
    with open(path_depth, mode='rb') as file: # b is important -> binary
        fileContent = file.read()
    x=np.fromstring(fileContent[20:],dtype=np.uint16) #first 20 digits have other information
    #Cada matriz de 424x512=217088 tiene delante 4 digitos de información (nº de evento, etc..) que hay que saltar. Son 1027 matrices
    c=0
    arrays_list=[]
    for i in range(0,2058,2):
        try:
            x1=x[4+108546*i:108546*(i+2)]
            #cv2.imwrite('D:\\TeDFES\\kinect_raw\\records\\record2\\prueba dat\\array_{}.jpeg'.format(c),x1.reshape(424,512))
            arrays_list.append(x1.reshape(424,512))
            c+=1
        except:pass
    return(arrays_list)

In [19]:
#EXAMPLE
path_depth='D:\\TeDFES\\kinect_raw\\records\\record2\\record2_Depth.dat'
arrays=read_Depth(path_depth)
#Depth of pixel 50,50 in the frame 30 in mm
arrays[200][50][50]

D:\Anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  after removing the cwd from sys.path.


2229

## Obtain real corrdinates X and Y from pixels

In [20]:
def obtain_real_x(x,depth, camera='IR'):
    if camera=='IR':
    #dimensiones= 512x424
    #distancia focal=3.657m (plano de proyección)
    #Angulo horizontal= 70 deg
    #Angulo vertical= 60.2 deg
        x_img_px=x-256
        
        #<____2.56m_____>
        #|              |
        #|             |
  #3.657m|            |
        #|           |
        #|          |
        #|         |
        #|        |
        #|35 deg |
        #<CAMERA>
        
        x_img=x_img_px*10
        x_real_max=depth*np.sin(np.deg2rad(35))/np.sin(np.deg2rad(55))
        x_real=x_img*x_real_max/2560
        return(x_real)
    
    if camera=='RGB':
    #dimensiones= 1920x1080
    #distancia focal=3.291m (plano de proyección)
    #Angulo horizontal= 84.7 deg
    #Angulo vertical= 54.4 deg
        x_img=x-960

        x_real_max=depth*np.sin(np.deg2rad(42))/np.sin(np.deg2rad(48))
        x_real=x_img*x_real_max/960
        return(x_real)
    
def obtain_real_y(y,depth, camera='IR'):
    
    if camera=='IR':
        #                          |
        #                        | |
        #                      |   |
        #                    |     |
        #                  |       |
        #                |         |2.12 m
        #              |           |
        #            |             |
        #          |               |
        # CAMERA |_30.1 deg________|
        #         <    3.657m     >  
        y=424-y
        y_img_px=y-212
        y_img=y_img_px*10
        y_real_max=depth*np.sin(np.deg2rad(30.1))/np.sin(np.deg2rad(60))
        y_real=y_img*y_real_max/2120
        return(y_real)
    
    if camera=='RGB':
        y=1080-y
        y_img=y-540

        y_real_max=depth*np.sin(np.deg2rad(22.5))/np.sin(np.deg2rad(67.5))
        y_real=y_img*y_real_max/540
        return(y_real)

In [22]:
#EXAMPLE
#three points in an image (pixels coordinates)
p1=(316,290)
p2=(308,250)
p3=(310,195)


n=257 #number of frame

y3=obtain_real_y(p3[1],arrays[n][p3[1]][p3[0]]) # y coordinate pixel and depth 
x3=obtain_real_x(p3[0],arrays[n][p3[1]][p3[0]]) #  x coordinate pixel and depth 
p3_real=[int(x3),int(y3)]
p3_real.append(arrays[n][p3[1]][p3[0]])         # depth

y2=obtain_real_y(p2[1],arrays[n][p2[1]][p2[0]])
x2=obtain_real_x(p2[0],arrays[n][p2[1]][p2[0]])
p2_real=[int(x2),int(y2)]
p2_real.append(arrays[n][p2[1]][p2[0]])

y1=obtain_real_y(p1[1],arrays[n][p1[1]][p1[0]])
x1=obtain_real_x(p1[0],arrays[n][p1[1]][p1[0]])
p1_real=[int(x1),int(y1)]
p1_real.append(arrays[n][p1[1]][p1[0]])

# real points (x,y,z), z=depth
print(p1_real,p2_real,p3_real)

[201, -262, 1230] [179, -131, 1263] [148, 46, 1007]
